<a href="https://colab.research.google.com/github/Alquimia-ai/intent-classification/blob/main/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Alquimia text classification training 🤗🤗

#### Setup credentials & dependencies

In [1]:
!pip install transformers==4.28.0 datasets evaluate accelerate ipywidgets jupyterlab_widgets  -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.6 MB/s eta 0:00:00


In [2]:
!git config --global credential.helper store

In [3]:
# Access token through https://huggingface.co/settings/tokens
from huggingface_hub import notebook_login

notebook_login()

In [4]:
## Load some important variables
import ipywidgets as widgets
from IPython.display import display

# Creating the widgets
test_size_slider = widgets.FloatSlider(
    value=0.2,
    min=0,
    max=1.0,
    step=0.05,
    description='Test Size:',
    continuous_update=False
)

number_of_labels_spinner = widgets.BoundedIntText(
    value=2,
    min=1,
    max=100,
    step=1,
    description='Number of Labels:'
)

label_column_name_text = widgets.Text(
    value='label',
    placeholder='Type something',
    description='Label Column:',
    disabled=False
)

text_column_name_text = widgets.Text(
    value='text',
    placeholder='Type something',
    description='Text Column:',
    disabled=False
)

# Button to set the variables
def set_variables(button):
    global test_size, number_of_labels, label_column_name, text_column_name
    test_size = test_size_slider.value
    number_of_labels = number_of_labels_spinner.value
    label_column_name = label_column_name_text.value
    text_column_name = text_column_name_text.value
    print(f"Set values: Test Size: {test_size}, Number of Labels: {number_of_labels}, Label Column: {label_column_name}, Text Column: {text_column_name}")

set_button = widgets.Button(description="Set Variables")
set_button.on_click(set_variables)

# Displaying the widgets
display(test_size_slider, number_of_labels_spinner, label_column_name_text, text_column_name_text, set_button)

FloatSlider(value=0.2, continuous_update=False, description='Test Size:', max=1.0, step=0.05)

BoundedIntText(value=2, description='Number of Labels:', min=1)

Text(value='label', description='Label Column:', placeholder='Type something')

Text(value='text', description='Text Column:', placeholder='Type something')

Button(description='Set Variables', style=ButtonStyle())

Set values: Test Size: 0.2, Number of Labels: 6, Label Column: sentiment, Text Column: question


In [5]:
try:
    print(f"Test Size: {test_size}")
    print(f"Number of Labels: {number_of_labels}")
    print(f"Label Column Name: {label_column_name}")
    print(f"Text Column Name: {text_column_name}")
except NameError:
    print("Variables not set yet. Please set them using the widgets and the 'Set Variables' button in the previous cell.")

Test Size: 0.2
Number of Labels: 6
Label Column Name: sentiment
Text Column Name: question


In [6]:
import ipywidgets as widgets
from IPython.display import display

# Create the FileUpload widget
upload = widgets.FileUpload(
    accept='.csv',  # Accept only .csv files
    multiple=False  # Do not allow multiple file uploads
)

display(upload)

FileUpload(value={}, accept='.csv', description='Upload')

In [7]:
import pandas as pd
import io

# Check if a file has been uploaded
if upload.value:
    # Retrieve uploaded file details
    uploaded_filename = next(iter(upload.value))
    content = upload.value[uploaded_filename]['content']

    # Decode content if it's in bytes or memoryview format
    if isinstance(content, (bytes, memoryview)):
        content = content.decode('utf-8')

    df = pd.read_csv(io.StringIO(content))
    print("File uploaded successfully!")
else:
    print("No file uploaded.")


File uploaded successfully!


In [8]:
## Show first three registers
df.head(3)

,annotation_id,annotator,created_at,id,lead_time,question,sentiment,updated_at
0,1120,1,2023-11-08T20:41:06.680849Z,1165,2.084,"I ordered two pairs of skis, 173cm, and they s...",postSale,2023-11-08T20:41:06.680863Z
1,283,1,2023-10-30T13:20:25.196662Z,287,2.081,"What does ""lithotripsy"" mean?",irrelevant,2023-10-30T13:20:25.196675Z
2,1295,1,2023-11-10T15:55:32.460520Z,1340,1.637,I bought a summer skirt for my daughter in a s...,postSale,2023-11-10T15:55:32.460532Z


### Define mapping

Here you have define a mapping so the model can be properly trained  lets see an example

```python
category_to_label = {
    'availability': 0,
    'irrelevant': 1,
    'post sale': 2,
    'invoice':3,
    'service':4,
    'pricing':5,
    'general':6,
    'cancelation policy':7,
    'cancel reservation':8
}
```

In [9]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def create_mapping_interactively():
    # Create text box for categories and labels
    category_text = widgets.Text(value='', placeholder='Enter category', description="Category:")
    label_text = widgets.IntText(value=0, description="Label:")
    add_button = widgets.Button(description="Add")
    generate_button = widgets.Button(description="Generate Dictionary")
    output = widgets.Output()

    # Temporary storage for categories and labels
    temp_storage = {}

    # Function to handle adding category and label
    def add_category_label(b):
        with output:
            clear_output(wait=True)
            temp_storage[category_text.value] = label_text.value
            print("Current Mapping:")
            for key, val in temp_storage.items():
                print(f"{key} -> {val}")
            category_text.value = ''  # Clear the category input box for new entry

    # Function to finalize the dictionary
    def generate_dictionary(b):
        with output:
            clear_output(wait=True)
            print("Final Dictionary:")
            print(temp_storage)
            global category_to_label
            category_to_label = temp_storage  # Saving the mapping to a global variable

    # Link button actions to respective functions
    add_button.on_click(add_category_label)
    generate_button.on_click(generate_dictionary)

    # Display the widgets
    display(category_text, label_text, add_button, generate_button, output)

create_mapping_interactively()


Text(value='', description='Category:', placeholder='Enter category')

IntText(value=0, description='Label:')

Button(description='Add', style=ButtonStyle())

Button(description='Generate Dictionary', style=ButtonStyle())

Output()

In [10]:
category_to_label

{'inventory': 0,
 'checkout': 1,
 'irrelevant': 2,
 'conversational': 3,
 'feedback': 4,
 'postSale': 5}

In [11]:
# Add the new 'label' column to the dataframe by mapping values from the 'category' column
df['label'] = df[label_column_name].replace(category_to_label)
df=df.drop('annotation_id',axis=1)
df=df.drop('annotator',axis=1)
df=df.drop('created_at',axis=1)
df=df.drop('id',axis=1)
df=df.drop('lead_time',axis=1)
df=df.drop('updated_at',axis=1)
df.head(3)

,question,sentiment,label
0,"I ordered two pairs of skis, 173cm, and they s...",postSale,5
1,"What does ""lithotripsy"" mean?",irrelevant,2
2,I bought a summer skirt for my daughter in a s...,postSale,5


### Train/Test split

In [12]:
from sklearn.model_selection import train_test_split


df_train,df_test=train_test_split(df,test_size=test_size)

### Convert to HuggingFace dataset

In [13]:
from datasets import Dataset

train_dataset=Dataset.from_pandas(df_train)
test_dataset=Dataset.from_pandas(df_test)

In [14]:
from transformers import AutoTokenizer
model_name='distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

(…)rt-base-uncased/resolve/main/config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

(…)bert-base-uncased/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)base-uncased/resolve/main/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [15]:
def preprocess_function(examples):
    return tokenizer(examples[text_column_name],truncation=True)

In [16]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test=test_dataset.map(preprocess_function,batched=True)

Map:   0%|          | 0/386 [00:00<?, ? examples/s]

Map:   0%|          | 0/97 [00:00<?, ? examples/s]

### PyTorch model

In [17]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=number_of_labels)

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [18]:
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np
from transformers import DataCollatorWithPadding

data_collator=DataCollatorWithPadding(tokenizer=tokenizer)

In [19]:
metric= evaluate.load("accuracy")

def compute_metrics(eval_pred):
  logits,labels=eval_pred
  predictions=np.argmax(logits,axis=-1)
  return metric.compute(predictions=predictions,references=labels)

In [20]:
import ipywidgets as widgets
from IPython.display import display

# Create a text input widget
model_name_widget = widgets.Text(
    value='',
    placeholder='Enter model name',
    description='Model Name:',
    disabled=False
)

# Create a button for confirmation
confirm_button = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='',  # 'success', 'info', 'warning', 'danger', or ''
    tooltip='Click to confirm the model name',
    icon='check'  # FontAwesome icon name without `fa-`
)

# Function to handle the input once the button is clicked
def on_button_click(button):
    global hf_model_name
    hf_model_name = model_name_widget.value
    print(f"Saved model name: {hf_model_name}")

# Attach the button click event to the function
confirm_button.on_click(on_button_click)

# Display the widget and button
display(model_name_widget, confirm_button)


Text(value='', description='Model Name:', placeholder='Enter model name')

Button(description='Confirm', icon='check', style=ButtonStyle(), tooltip='Click to confirm the model name')

Saved model name: rowdy-store/intent-classification


In [21]:
training_args = TrainingArguments(
    hub_model_id=hf_model_name,
    output_dir="./output",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    eval_dataset=tokenized_test
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/rowdy-store/intent-classification into local empty directory.


Download file pytorch_model.bin:   0%|          | 8.00k/255M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.93k/3.93k [00:00<?, ?B/s]

Download file runs/Nov03_19-40-10_0bc8ca3987aa/events.out.tfevents.1699040423.0bc8ca3987aa.175.0: 100%|#######…

Download file runs/Nov03_19-40-10_0bc8ca3987aa/1699040423.2038674/events.out.tfevents.1699040423.0bc8ca3987aa.…

...

Clean file runs/Nov03_19-40-10_0bc8ca3987aa/1699040423.2038674/events.out.tfevents.1699040423.0bc8ca3987aa.175…

Clean file runs/Nov07_20-59-01_d723ad21d570/events.out.tfevents.1699390850.d723ad21d570.237.0:  15%|#4        …

Clean file runs/Nov07_20-59-01_d723ad21d570/1699390850.2936895/events.out.tfevents.1699390850.d723ad21d570.237…

Clean file runs/Nov03_19-40-10_0bc8ca3987aa/events.out.tfevents.1699040423.0bc8ca3987aa.175.0:  15%|#4        …

Clean file pytorch_model.bin:   0%|          | 1.00k/255M [00:00<?, ?B/s]

In [22]:
### Fine tune model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.621400,0.214365,0.907216
2,0.195600,0.141846,0.927835
3,0.122400,0.277322,0.948454
4,0.045800,0.118609,0.958763
5,0.019800,0.106345,0.969072


TrainOutput(global_step=245, training_loss=0.2010355105205458, metrics={'train_runtime': 572.5495, 'train_samples_per_second': 3.371, 'train_steps_per_second': 0.428, 'total_flos': 16204609462032.0, 'train_loss': 0.2010355105205458, 'epoch': 5.0})

In [23]:
### Upload to huggingface

trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/255M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/3.93k [00:00<?, ?B/s]

Upload file runs/Nov10_18-06-50_042572c6fe34/events.out.tfevents.1699639707.042572c6fe34.126.0:   0%|         …

Upload file runs/Nov10_18-06-50_042572c6fe34/1699639707.2172892/events.out.tfevents.1699639707.042572c6fe34.12…

To https://huggingface.co/rowdy-store/intent-classification
   ef8d0cb..47e2427  main -> main

   ef8d0cb..47e2427  main -> main

To https://huggingface.co/rowdy-store/intent-classification
   47e2427..693ca2f  main -> main

   47e2427..693ca2f  main -> main



'https://huggingface.co/rowdy-store/intent-classification/commit/47e2427127a245b7d373a0cc89f9432118f65a96'